# 🏙️ Chapter 11: 3D Vectorization & Modeling

Processing raw point clouds into lightweight vector formats (like Shapefiles, GeoJSON, or CAD models) is the ultimate goal of many geospatial workflows. This process is called **Vectorization**.

**Workflow:**
1.  **Load** LiDAR data (`.las`/`.laz`).
2.  **Filter** ground points to isolate objects (buildings, trees).
3.  **Cluster** individual objects.
4.  **Trace** boundaries (2D outlines) using Alpha Shapes.
5.  **Extrude** to 3D models.

In [ ]:
import numpy as np
import laspy
import open3d as o3d
import matplotlib.pyplot as plt
from shapely.geometry import MultiPoint, Polygon
from shapely.ops import unary_union
import alphashape

## 1. Load LiDAR Data

We use `laspy` to handle LAS/LAZ files, standard in aerial surveying.

In [ ]:
# Load file
filename = "../DATA/single_building.laz"

try:
    las = laspy.read(filename)
    print(f"Loaded LAS file with {len(las.points)} points.")
    
    # Extract XYZ
    points = np.stack([las.x, las.y, las.z], axis=0).transpose((1, 0))
    
    # Create Open3D cloud for visualization
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    
    # Color by Z (height)
    colors = plt.get_cmap("terrain")( (points[:, 2] - points[:, 2].min()) / (points[:, 2].max() - points[:, 2].min()) )
    pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])
    
    o3d.visualization.draw_geometries([pcd], window_name="LiDAR Data")
    
except FileNotFoundError:
    print("LAS file not found. Creating dummy data.")
    pcd = o3d.geometry.TriangleMesh.create_box(width=10, height=10, depth=5).sample_points_poisson_disk(5000)
    points = np.asarray(pcd.points)

## 2. Filtering Non-Ground Points

We typically want buildings. If the data is classified (LAS classification 6=Building), we use that. If not, we use a geometric filter (CSF or height threshold).

For this tutorial, let's assume we filter by height or existing classification.

In [ ]:
# Example: Select points classified as 'Building' (Class 6)
try:
    building_mask = las.classification == 6
    if np.sum(building_mask) > 0:
        building_points = points[building_mask]
        print(f"Found {len(building_points)} building points.")
    else:
        print("No building class found. Using height threshold.")
        building_points = points[points[:, 2] > points[:, 2].min() + 2.0]
except:
    building_points = points

pcd_buildings = o3d.geometry.PointCloud()
pcd_buildings.points = o3d.utility.Vector3dVector(building_points)
pcd_buildings.paint_uniform_color([1, 0.7, 0]) # Orange buildings
o3d.visualization.draw_geometries([pcd_buildings], window_name="Buildings")

## 3. Clustering per Building

We use DBSCAN to separate distinct buildings.

In [ ]:
# DBSCAN
labels = np.array(pcd_buildings.cluster_dbscan(eps=1.5, min_points=50, print_progress=True))
max_label = labels.max()
print(f"Detected {max_label + 1} buildings.")

colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
pcd_buildings.colors = o3d.utility.Vector3dVector(colors[:, :3])
o3d.visualization.draw_geometries([pcd_buildings])

## 4. Vectorizing footprints (2D)

For each cluster, we project points to 2D (XY plane) and compute the **Concave Hull** (Alpha Shape) to get the footprint.

In [ ]:
def get_footprint(points_3d, alpha=0.9):
    # Project to 2D
    points_2d = points_3d[:, :2]
    # Compute Alpha Shape
    try:
        alpha_shape = alphashape.alphashape(points_2d, alpha)
        return alpha_shape
    except:
        return None

# Process first 5 buildings
for i in range(min(5, max_label + 1)):
    mask = labels == i
    cluster_points = building_points[mask]
    
    poly = get_footprint(cluster_points, alpha=0.5)
    
    if poly and not poly.is_empty:
        x, y = poly.exterior.xy
        plt.fill(x, y, alpha=0.5, label=f'Building {i}')
        plt.plot(x, y, 'k-')

plt.title("Extracted Footprints")
plt.axis('equal')
plt.show()